### 압축해제

In [1]:
import zipfile
import os
import glob
from PIL import Image
import json

# zip 파일 압축 해제
def unzip_specific_count(zip_files, output_folder, count):
    for zip_path in zip_files[:count]:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_folder)

# png to jpg
def convert_png_to_jpg(png_path, output_folder):
    with Image.open(png_path) as img:
        base_name = os.path.basename(png_path).rsplit('.', 1)[0]
        output_path = os.path.join(output_folder, f"{base_name}.jpg")
        img.convert('RGB').save(output_path, 'JPEG')

In [15]:
##### 이미지파일 

# 경로설정
img_train_root_dir = "D:\\Ddrive\\3rd\\data\\01.데이터\\Training\\01.원천데이터"
img_train_output_dir = "D:\\Ddrive\\3rd\\data_ex\\training\\images"

# img_val_root_dir = "D:\\Ddrive\\3rd\\data\\01.데이터\\Validation\\01.원천데이터"
# img_val_output_dir = "D:\\Ddrive\\3rd\\data_ex\\validation\\images"
# img_train_root_dir = img_val_root_dir
# img_train_output_dir = img_val_output_dir

In [6]:
# 압축 파일 목록 가져오기
train_zip_files = sorted(glob.glob(os.path.join(img_train_root_dir, "*.zip")))

# 100개씩 ZIP 파일 압축 해제
chunk_size = 100
for idx in range(0, len(train_zip_files), chunk_size):
    unzip_specific_count(train_zip_files[idx: idx + chunk_size], img_train_output_dir, chunk_size)

    # PNG 파일 목록 가져오기
    png_files = glob.glob(os.path.join(img_train_output_dir, '*.png'))

    # PNG를 JPG로 변환
    for png_path in png_files:
        convert_png_to_jpg(png_path, img_train_output_dir)
        os.remove(png_path)  # PNG 파일 삭제

c:\Python\Python39\lib\site-packages\PIL\Image.py:3157: DecompressionBombWarning: Image size (133505320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [17]:
##### 라벨링 파일

lbl_train_root_dir = "D:\\Ddrive\\3rd\\data\\01.데이터\\Training\\02.라벨링데이터"
lbl_train_output_dir = "D:\\Ddrive\\3rd\\data_ex\\Training\\labels"

# lbl_val_root_dir = "D:\\Ddrive\\3rd\\data\\01.데이터\\Validation\\02.라벨링데이터"
# lbl_val_output_dir = "D:\\Ddrive\\3rd\\data_ex\\Validation\\labels"

# lbl_train_root_dir = lbl_val_root_dir
# lbl_train_output_dir = lbl_val_output_dir

In [5]:
# 압축 파일 목록 가져오기
train_zip_files = sorted(glob.glob(os.path.join(lbl_train_root_dir, "*.zip")))

# 100개씩 ZIP 파일 압축 해제
chunk_size = 100
for idx in range(0, len(train_zip_files), chunk_size):
    unzip_specific_count(train_zip_files[idx: idx + chunk_size], img_train_output_dir, chunk_size)


### 파일 조회

In [5]:
img_train_output_dir = "D:\\Ddrive\\3rd\\data_ex\\training\\images"
img_val_output_dir = "D:\\Ddrive\\3rd\\data_ex\\validation\\images"

lbl_train_output_dir = "D:\\Ddrive\\3rd\\data_ex\\Training\\labels"
lbl_val_output_dir = "D:\\Ddrive\\3rd\\data_ex\\Validation\\labels"


In [9]:
##### JSON 파일 하나 읽기
extracted_label_files = os.listdir(lbl_train_output_dir)
filename = extracted_label_files[5]
json_file_path = os.path.join(lbl_train_output_dir+'/'+filename)
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
data

{'filename': '0001_1992_03_00000006_F',
 'id': 1,
 'birth': 1992,
 'age_now': 31,
 'age_past': 3,
 'format': 'png',
 'imgsize': '6.058MB',
 'width': 2268,
 'height': 4032,
 'device': 'film',
 'gender': 'male',
 'annotation': [{'box': {'x': 891.2232817154722,
    'y': 1053.9795875544532,
    'w': 588.0784269947857,
    'h': 537.6325742232132},
   'landmark': [[1199.6512395709426, 1300.9796508392835],
    [1077.2043124256556, 1209.0285228620487],
    [1305.2694535496466, 1185.7092774229955],
    [1107.4994950800155, 1455.4955492450852],
    [1279.2178206070441, 1462.0890281459724]]}]}

In [10]:
##### 이미지 파일 하나 조회

from PIL import Image, ImageDraw, ImageFont

# 이미지 파일 경로
extracted_label_files = os.listdir(img_train_output_dir)
image_file_path = os.path.join(img_train_output_dir+'/'+extracted_label_files[5])

# 이미지 열기
img = Image.open(image_file_path)

# 이미지 위에 박스 좌표 그리기
draw = ImageDraw.Draw(img)

for annotation in data['annotation']:
    box = annotation['box']
    x, y, w, h = box['x'], box['y'], box['w'], box['h']
    draw.rectangle([x, y, x + w, y + h], outline="red", width=2)

    # landmark 그리기
    for landmark_point in annotation['landmark']:
        lx, ly = landmark_point
        draw.ellipse([lx - 2, ly - 2, lx + 2, ly + 2], outline="blue", width=2)

    # 박스 좌표 표시
    font = ImageFont.truetype("arial.ttf", 12)  # 폰트 및 크기 설정
    draw.text((x, y), f"x: {x}, y: {y}", fill="red", font=font)
    draw.text((x, y + h), f"w: {w}, h: {h}", fill="red", font=font)
# 이미지 표시
img.show()
